In [ ]:
import numpy as np
import pandas as pd

def Mean(bag):
    return sum(bag) / float(len(bag))


def Variance(bag):
    mean = Mean(bag)
    return sum([ pow(i - mean, 2) for i in bag ]) / float(len(bag) - 1)


def normalDFCof(bag):
    return (Mean(bag), Variance(bag))


def normalDF(x, mean, var):
    return np.exp(-pow(x - mean, 2) / (2 * var)) / np.sqrt(2 * np.pi * var)


def classify(x, training):
    # group training data by match winner
    grouped = training.groupby('Match Winner')
    fst, snd = grouped.get_group(0), grouped.get_group(1)

    # remove Match Winner column
    fst = fst[fst.columns[:-1]]
    snd = snd[snd.columns[:-1]]

    # create normal density function for each column
    fstDFCof = [ normalDFCof(fst[col]) for col in fst.columns ]
    sndDFCof = [ normalDFCof(snd[col]) for col in snd.columns ]

    # calculate probability of each group
    fstPr, sndPr = 1, 1
    for i, v in enumerate(x):
        fstPr *= normalDF(v, fstDFCof[i][0], fstDFCof[i][1])
        sndPr *= normalDF(v, sndDFCof[i][0], sndDFCof[i][1])

    return {0: fstPr, 1: sndPr}


df = pd.read_csv('ausopen.csv')

grouped = df.groupby('Match Winner')
fst, snd = grouped.get_group(0), grouped.get_group(1)

training = fst.head(5).append(snd.head(5))

x = snd[6:7].values.tolist()[0][:-1]

scores = classify(x, training)

print(scores)

if np.maximum(scores[0], scores[1]) == scores[0]:
    print("First team")
else:
    print("Second team")